# Generic Rule Induction Notebook

Continuoulsy refined.

In [1]:
#%store -z acc_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [2]:
from config import config_dict

# document config order
CONFIG = config_dict["CONFIG14"]
print('Proceed with configuration:', CONFIG["NAME"])

[('CONFIG1', {'NAME': 'binary-churn-quantile-brcg', 'DATA_SET': '../data/churn_prob_out_35.csv', 'DATA_TYPES': {'Children': <class 'float'>, 'RatePlan': <class 'str'>}, 'DROP': ['Id', 'pChurn', '3_Class', '5_Class', 'is_test_set'], 'MODE': 'PREDICTIVE', 'TRAIN_TEST_SPLIT': 0.3, 'BINARIZER': 'QUANTILE', 'ALGO': 'BRCG', 'TARGET_LABEL': 'CHURN', 'TYPE': 'BINARY', 'EXAMPLE_FEATURE': 'Est Income', 'POS_CLASS': 'T', 'BASELINE': True, 'USECASE': None}), ('CONFIG2', {'NAME': 'continuous-churn', 'DATA_SET': '../data/churn_prob_out_35.csv', 'DATA_TYPES': {'Children': <class 'float'>, 'RatePlan': <class 'str'>}, 'DROP': ['Id', 'CHURN', '3_Class', '5_Class', 'is_test_set'], 'MODE': 'PREDICTIVE', 'TRAIN_TEST_SPLIT': 0.3, 'BINARIZER': 'QUANTILE', 'ALGO': 'GLRM', 'TARGET_LABEL': 'pChurn', 'TYPE': 'CONTINUOUS', 'EXAMPLE_FEATURE': 'Est Income', 'POS_CLASS': None, 'BASELINE': True, 'USECASE': None}), ('CONFIG3', {'NAME': 'bike-demand', 'DATA_SET': '../data/SeoulBikeData.csv', 'DATA_TYPES': {'Rented Bike

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *





# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


Importing dev version v0.981 of RIPPER


### Data

In [4]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1 
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      8760 non-null   object 
 1   Hour                      8760 non-null   int64  
 2   Temperature(C)            8760 non-null   float64
 3   Humidity(%)               8760 non-null   int64  
 4   Wind speed (m/s)          8760 non-null   float64
 5   Visibility (10m)          8760 non-null   int64  
 6   Dew point temperature(C)  8760 non-null   float64
 7   Solar Radiation (MJ/m2)   8760 non-null   float64
 8   Rainfall(mm)              8760 non-null   float64
 9   Snowfall (cm)             8760 non-null   float64
 10  Seasons                   8760 non-null   object 
 11  Holiday                   8760 non-null   object 
 12  Functioning Day           8760 non-null   object 
 13  Target                    8760 non-null   int64  
dtypes: float

,Date,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,Target
0,01/12/2017,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,0
1,01/12/2017,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,0
2,01/12/2017,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,0
3,01/12/2017,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,0
4,01/12/2017,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,0


In [5]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

0    5238
1    3522
Name: Target, dtype: int64


### Train, Test Split

In [6]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (6132, 13) (6132,)
Test: (2628, 13) (2628,)


### Reference Performance

In [7]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [8]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=True) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6132 entries, 1444 to 7270
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      6132 non-null   object 
 1   Hour                      6132 non-null   int64  
 2   Temperature(C)            6132 non-null   float64
 3   Humidity(%)               6132 non-null   int64  
 4   Wind speed (m/s)          6132 non-null   float64
 5   Visibility (10m)          6132 non-null   int64  
 6   Dew point temperature(C)  6132 non-null   float64
 7   Solar Radiation (MJ/m2)   6132 non-null   float64
 8   Rainfall(mm)              6132 non-null   float64
 9   Snowfall (cm)             6132 non-null   float64
 10  Seasons                   6132 non-null   object 
 11  Holiday                   6132 non-null   object 
 12  Functioning Day           6132 non-null   object 
dtypes: float64(6), int64(3), object(4)
memory usage: 670.7+ KB


,Date,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
1444,30/01/2018,4,-11.1,50,1.2,1986,-19.4,0.00,0.0,0.0,Winter,No Holiday,Yes
1652,07/02/2018,20,-5.8,44,2.1,1994,-16.1,0.00,0.0,0.0,Winter,No Holiday,Yes
7496,09/10/2018,8,11.4,66,0.8,1991,5.2,0.18,0.0,0.0,Autumn,Holiday,No
1893,17/02/2018,21,-2.3,38,2.3,2000,-14.7,0.00,0.0,0.0,Winter,Holiday,Yes
3880,11/05/2018,16,19.1,54,3.2,542,9.5,0.94,0.0,0.0,Spring,No Holiday,Yes


In [9]:
#x_train_bin = x_train_bin.reset_index(drop = True)
y_train.value_counts()

0    3646
1    2486
Name: Target, dtype: int64

### Rule Induction

In [10]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                     max_card=2, # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for RIPPER
Training time: 2.0729990005493164


### Evaluation

In [11]:

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.8900304414003044
Balanced accuracy: 0.8844560641043053
Precision: 0.8622696411251213
Recall: 0.8581081081081081
F1 0.860183841315917
ConfusionMatrix [[1450  142]
 [ 147  889]]
F-2 0.8589371980676327


In [12]:
#XGboost for Binary Classification
# storing the results in a list
acc_list = []
%store -r acc_list
if CONFIG['TYPE'] == 'BINARY' and CONFIG['BASELINE'] == True:
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('F-2', fbeta_score(y_test, preds, pos_label=CONFIG['POS_CLASS'], beta= 2))
    acc_list.append(accuracy_score(y_test, preds))
    %store acc_list

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields Date, Seasons, Holiday, Functioning Day

In [13]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    print(estimator.rule_list_to_pretty_string())
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        print(f"Antecedents Length Rule {i}:" , an)

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Rule count: 31
Rule set:

if {
	(Solar Radiation (MJ/m2) >= 0.35 and Functioning Day == Yes and Dew point temperature(C) <= 17.4 and Temperature(C) >= 21.4) or
	(Rainfall(mm) <= 0.1 and Dew point temperature(C) >= 9.1 and Dew point temperature(C) <= 10.1 and Temperature(C) <= 15.2 and Solar Radiation (MJ/m2) >= 0.06 and Temperature(C) >= 12.5) or
	(Temperature(C) <= 21.2 and Solar Radiation (MJ/m2) >= 1.16 and Functioning Day == Yes and Temperature(C) >= 13.6 and Dew point temperature(C) >= 7.7) or
	(Solar Radiation (MJ/m2) >= 0.01 and Seasons == Spring and Rainfall(mm) <= 0.0 and Dew point temperature(C) >= 9.4 and Dew point temperature(C) <= 13.2 and Temperature(C) >= 14.9) or
	(Temperature(C) >= 21.1 and Dew point temperature(C) <= 20.0 and Functioning Day == Yes and Temperature(C) <= 26.9 and Solar Radiation (MJ/m2) >= 0.82) or
	(Solar Radiation (MJ/m2) >= 0.02 and Solar Radiation (MJ/m2) <= 0.63 and Temperature(C) >= 28.5) or
	(Hour == 21 and Temperature(C) >= 10.7 and Functioning